In [1]:
from skdata.mnist.views import OfficialVectorClassification
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
import time

from itertools import *
%matplotlib inline

In [2]:
# Generate Audio Data
def generate_audio_wave(frequency=440.0, framerate=44100, amplitude=0.5):
    i = 0
    while True:
        yield float(amplitude) + math.cos(2.0*math.pi*float(frequency)*(float(i)/float(framerate)))
        i += 1

In [3]:
def generate_seq(seq_leng):
    
    # Generate audio wave
    wave = generate_audio_wave(amplitude=1)
    
    # Generate function values
    data_seq = []
    for i in range(seq_leng):
        data_seq.append(wave.next())
    return data_seq

In [4]:
def generate_window(data, n_int, window_size):

    input_data=[]
    output_data=[]
    
    for i in range(0, n_int - window_size, 1):
        seq_in = data[i:i + window_size]
        seq_out = data[i + window_size]
        input_data.append(seq_in)
        output_data.append(seq_out)
    return input_data, output_data

In [5]:
data = generate_seq(100)

# Input, Output
input_data, output_data = generate_window(data, 100, 10)

In [ ]:
writer = tf.python_io.TFRecordWriter("sin.tfrecords")

In [ ]:
for i in tqdm(range(len(input_data))):
    features = input_data[i]
    label = output_data[i]
    # construct the Example proto boject
    example = tf.train.Example(
        # Example contains a Features proto object
        features=tf.train.Features(
          # Features contains a map of string to Feature proto objects
          feature={
            # A Feature contains one of either a float_list,
            # float_list, or bytes_list
            'label': tf.train.Feature(
                float_list=tf.train.FloatList(value=[label])),
            'seq': tf.train.Feature(
                float_list=tf.train.FloatList(value=features)),
    }))
    
    # use the proto object to serialize the example to a string
    serialized = example.SerializeToString()
    # write the serialized object to disk
    writer.write(serialized)

In [6]:
# Network Parameters
num_input = 10 # MNIST data input (img shape: 28*28)
num_hidden = 3 # hidden layer num of features
num_batch = 1

# Parameters
learning_rate = 0.001

In [7]:
filename_queue = tf.train.string_input_producer(["sin.tfrecords"], num_epochs=None)

In [8]:
reader = tf.TFRecordReader()

In [9]:
_, serialized_example = reader.read(filename_queue)

In [10]:
features = tf.parse_single_example(
        serialized_example,
        features={
            # We know the length of both fields. If not the
            # tf.VarLenFeature could be used
            'label': tf.FixedLenFeature([1], tf.float32),
            'seq': tf.FixedLenFeature([10], tf.float32)
        })

In [11]:
label = features['label']
seq = features['seq']

In [12]:
print seq.get_shape()
print label.get_shape()

(10,)
(1,)


In [13]:
# groups examples into batches randomly
seq_batch, labels_batch = tf.train.shuffle_batch(
    [seq, label], batch_size=num_batch,
    capacity=2000,
    min_after_dequeue=1000)

In [14]:
print seq_batch.get_shape()
print labels_batch.get_shape()

(1, 10)
(1, 1)


In [15]:
seq_batch = tf.reshape(seq_batch, [num_batch, num_input, 1])
labels_batch = tf.reshape(labels_batch, [num_batch, 1])

In [16]:
print seq_batch.get_shape()
print labels_batch.get_shape()

(1, 10, 1)
(1, 1)


In [17]:
cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True)
val, state = tf.nn.dynamic_rnn(cell, seq_batch, dtype=tf.float32)

In [18]:
print cell
print val.get_shape()
print state

(1, 10, 3)
LSTMStateTuple(c=<tf.Tensor 'RNN/while/Exit_2:0' shape=(?, 3) dtype=float32>, h=<tf.Tensor 'RNN/while/Exit_3:0' shape=(?, 3) dtype=float32>)


In [19]:
val_test = tf.transpose(val, [1, 0, 2])

In [20]:
val_test

<tf.Tensor 'transpose_1:0' shape=(10, 1, 3) dtype=float32>

In [ ]:
val = tf.transpose(val, [1, 0, 2])
'''
    Take the last column instead of last row.
'''
last = tf.gather(val, int(val.get_shape()[0]) - 1)

weight = tf.Variable(tf.truncated_normal([num_hidden, 1]))
bias = tf.Variable(tf.truncated_normal([1]))

# Functions definition 
pred = tf.matmul(last, weight) + bias

# Define loss and optimizer
cost = tf.sqrt(tf.reduce_mean(tf.square(labels_batch - pred)), name='RMSE')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(labels_batch, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
last

In [ ]:
pred

In [ ]:
weight.get_shape()

In [ ]:
val.get_shape()

In [ ]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)
tf.train.start_queue_runners(sess=sess)

In [ ]:
for i in range(1000):
  # pass it in through the feed_dict
  _, loss_val = sess.run([optimizer, cost])
  print loss_val

In [ ]:
input_data[1]

In [ ]:
output_data[1]

In [ ]:
# Train predicted
test = np.array(input_data[:5])
test = test.reshape(5, 10, 1)
pred = sess.run(pred ,{seq_batch: test})
print pred

In [ ]:
output_data[:5]